# 유사도 기법
- 사용자와 아이템관 관계를 파악하여 사용자 유사도를 계산해본다.


#### 1. 코사인 유사도
코사인 유사도(― 類似度, 영어: cosine similarity)는 내적 공간의 두 벡터간 각도의 코사인 값을 이용하여 측정된 벡터간의 유사한 정도를 의미하며 코사인 유사도는 다차원에도 적용이 가능해 흔히 다차원의 양수 공간에서의 유사도 측정에 자주 이용된다. 그리고 코사인 유사도가 널리 사용되는 이유 중 하나는 이것이 양수 공간이라는 조건만 만족하면 얼마나 많은 차원 공간에서든지 거리를 측정하는 것이 가능하기 때문이다.
두 벡터의 코사인 값은 유클리디안 스칼라곱 공식에서 유도할 수 있다.
(aㆍb) = ||a||＊||b||＊cos(θ)
속성 A, B의 벡터값이 각각 주어졌을 때, 코사인 유사도 cos(θ)는 벡터의 스칼라곱과 크기로 다음과 같이 표현할 수 있다.
similarity = cos(θ) = (AㆍB) / (||A||＊||B||)
따라서 사용자의 코사인 유사도를 구하기 위해서는 사용자를 행으로 아이템을 열로 두고 사용자별 벡터를 생성해 이용한다.

#### 2. 자카드 유사도
자카드 유사도는 두 사용자간 아이템 사이의 합집합에 대한 교집합의 비율로 수식은 다음과 같다. 
J(A,B)=|A∩B||A∪B| (0<= J(A,B) <= 1)
코사인 유사도와 마찬가지로 사용자를 행으로 아이템을 열로 사용자별 벡터를 생성한다.

#### 참고
- http://bab2min.tistory.com/558
- http://rstatistics.tistory.com/29
- https://grouchy-sms.github.io/2017/09/28/Algorithm/basic/jaccard_similarity/
- https://zetawiki.com/wiki/Jaccard_%EC%9C%A0%EC%82%AC%EC%84%B1_%EC%A7%80%EC%88%98
- https://code.i-harness.com/ko/q/11921a4
- http://www.fun-coding.org/recommend_basic3.html

### 0. 데이터
다음은 5명의 과일 구매자가 구매한 과일 이력이다. 그렇다면 이 구매한 과일을 이용해 사용자간 유사도를 계산해보자.

In [1]:
data_dict = {
    'A': ['사과', '배', '포도', '복숭아', '수박', '참외'],
    'B': ['배', '포도', '수박'],
    'C': ['배', '포도', '수박', '오렌지', '자몽'],
    'D': ['배', '포도', '자몽'],
    'E': ['배', '포도', '수박', '수박', '오렌지', '자몽']
}    

### 1. 사용자-아이템 행렬 생성하기

In [2]:
import pandas as pd
import numpy as np

# 행 인덱스를 구한다.
def getRowIndex(dataType_dict):
    rowIndexList = [rowIndex for rowIndex in dataType_dict]
    return rowIndexList
# print(1, getRowIndex(data_dict))
rowIndexList = getRowIndex(data_dict)

# 열 인덱스를 구한다.
def getColumnIndex(dataType_dict):
    columnIndexList = [e for index in getRowIndex(data_dict) for e in dataType_dict[index]]
    return list(set(columnIndexList))
# print(2, getColumnIndex(data_dict))

# 아래와 같이도 열 인덱스를 가지고 올수 있음
# def getColumnsIndex(doubled_list):
#     ColumnsIndex = [e for list in doubled_list for e in list]
#     return list(set(ColumnsIndex))


# 사용자-아이템 데이터 프레임 만들기 -> 0, 1로 표현
def getUserItemMatrix(dataType_dict):
    rowIndexList = getRowIndex(dataType_dict)
    columnIndexList = getColumnIndex(dataType_dict)
    
    user_item_matrix = []
    for rowIndex in rowIndexList:
        tmp_list = []
        for columnIndex in columnIndexList:
            # 열 인덱스 값과 같으면 1 아니면 0
            tmp_list.append(1) if dataType_dict[rowIndex].count(columnIndex) else tmp_list.append(0)
            
        user_item_matrix.append(tmp_list)
    user_item_matrix_df = pd.DataFrame(user_item_matrix, columns=columnIndexList, index=rowIndexList)
    return user_item_matrix_df


# 사용자-아이템 데이터 프레임
user_item_matrix_df = getUserItemMatrix(data_dict)
print(user_item_matrix_df)
# 사용자-아이템 데이터 프레임을 행렬로 변환
user_item_matrix_matrix = user_item_matrix_df.values
print(user_item_matrix_matrix)

   포도  사과  배  자몽  오렌지  수박  참외  복숭아
A   1   1  1   0    0   1   1    1
B   1   0  1   0    0   1   0    0
C   1   0  1   1    1   1   0    0
D   1   0  1   1    0   0   0    0
E   1   0  1   1    1   1   0    0
[[1 1 1 0 0 1 1 1]
 [1 0 1 0 0 1 0 0]
 [1 0 1 1 1 1 0 0]
 [1 0 1 1 0 0 0 0]
 [1 0 1 1 1 1 0 0]]


### 2 코사인 유사도 계산하기

In [3]:
import math


# 소수 4번째 자리에서 반올림하는 함수
def setFormat(value):
    value =  round(value, 3)
    return value
# print(setFormat(0.70710678118654757))


# 두 벡터의 코사인 유사도를 계산하는 함수
def getCosineSimilarity(row1,row2):
    vec1 = np.array(row1)
    vec2 = np.array(row2)
    numerator = np.dot(vec1.T, vec2) # 두 벡터의 내적 값은 분자로
    sumSquaredVecElement1 = sum([v1**2 for v1 in vec1])
    sumSquaredVecElement2 = sum([v2**2 for v2 in vec2])
    denominator = math.sqrt(sumSquaredVecElement1) * math.sqrt(sumSquaredVecElement2) # 각 벡터의 제곱값의 합의 루트의 곱은 분모로
    cosineSimilarity = numerator/ denominator
    return cosineSimilarity

# print(user_item_matrix_matrix[0], user_item_matrix_matrix[1])
# print(getCosineSimilarity(user_item_matrix_matrix[0], user_item_matrix_matrix[1]))


# 모든 데이터에 대한 사용자별 코사인 유사도 계산 결과로 구성된 행렬을 반환한다.
def getCosineSimilarityMatrix(user_item_matrix_df):
    user_item_matrix_array = np.array(user_item_matrix_df)
    cosineSimilarityMatrix = []
    for i in range(len(user_item_matrix_array)):
        rowCosineSimilarity = []
        for j in range(len(user_item_matrix_array)):
            row1 = user_item_matrix_array[i]
            row2 = user_item_matrix_array[j]
            rowCosineSimilarity.append(getCosineSimilarity(row1,row2)) if i != j else rowCosineSimilarity.append(1)
        cosineSimilarityMatrix.append(rowCosineSimilarity)
    return cosineSimilarityMatrix

cosineSimilarityMatrix = getCosineSimilarityMatrix(user_item_matrix_df)
cosineSimilarityMatrix_df = pd.DataFrame(cosineSimilarityMatrix, index=rowIndexList, columns=rowIndexList)
print(cosineSimilarityMatrix_df)

          A         B         C         D         E
A  1.000000  0.707107  0.547723  0.471405  0.547723
B  0.707107  1.000000  0.774597  0.666667  0.774597
C  0.547723  0.774597  1.000000  0.774597  1.000000
D  0.471405  0.666667  0.774597  1.000000  0.774597
E  0.547723  0.774597  1.000000  0.774597  1.000000


### 3. 자카드 유사도

#### 3-1. 사용자간 아이텐 집합을 이용한  자카드 유사도

In [4]:
# a.intersection(b)은 a와 b의 교집합을 구하는 파이썬 내장함수이다. 이를 사용하기 위해서는 리스트형을 집합형(set)으로 변환해야한다.
def getJaccardSimilarity(row1, row2):
    s_row1 = set(row1)
    s_row2 = set(row2)
    numerator = s_row1.intersection(s_row2)
    denominator = s_row1.union(s_row2)
    jaccardSimilarity = len(numerator) / len(denominator)
    return jaccardSimilarity


def getJaccardSimilarityMatrix(data_dict):
    jaccardSimilarityMatrix = []
    for i in data_dict:
        rowjaccardSimilarity = []
        for j in data_dict:
            row1 = data_dict[i]
            row2 = data_dict[j]
            rowjaccardSimilarity.append(getJaccardSimilarity(row1,row2))
        jaccardSimilarityMatrix.append(rowjaccardSimilarity)
    return jaccardSimilarityMatrix


JaccardSimilarityMatrix = getJaccardSimilarityMatrix(data_dict)
JaccardSimilarityMatrix_df = pd.DataFrame(JaccardSimilarityMatrix, index=rowIndexList, columns=rowIndexList)
print(JaccardSimilarityMatrix_df) 

          A    B      C         D      E
A  1.000000  0.5  0.375  0.285714  0.375
B  0.500000  1.0  0.600  0.500000  0.600
C  0.375000  0.6  1.000  0.600000  1.000
D  0.285714  0.5  0.600  1.000000  0.600
E  0.375000  0.6  1.000  0.600000  1.000


#### 3-2. 사용자간 아이템 행렬을 이용해 자카드 유사도를 계산하는 함수

In [5]:
# a.intersection(b)은 a와 b의 교집합을 구하는 파이썬 내장함수이다. 이를 사용하기 위해서는 리스트형을 집합형(set)으로 변환해야한다.
def getJaccardSimilarity(row1, row2):
    interection = 0
    union = 0
    for x, y in zip(row1, row2):
        if x + y == 2:
            interection += 1 
        if x + y == 2 or x + y != 0:
            union += 1
    numerator = interection
    denominator = union
    jaccardSimilarity = numerator / denominator
    return jaccardSimilarity


def getJaccardSimilarityMatrix(user_item_matrix_df):
    user_item_matrix_array = np.array(user_item_matrix_df)
    jaccardSimilarityMatrix = []
    for i in range(len(user_item_matrix_array)):
        rowjaccardSimilarity = []
        for j in range(len(user_item_matrix_array)):
            row1 = user_item_matrix_array[i]
            row2 = user_item_matrix_array[j]
            rowjaccardSimilarity.append(getJaccardSimilarity(row1,row2)) if i != j else rowjaccardSimilarity.append(1)
        jaccardSimilarityMatrix.append(rowjaccardSimilarity)
    return jaccardSimilarityMatrix

JaccardSimilarityMatrix = getJaccardSimilarityMatrix(user_item_matrix_df)
JaccardSimilarityMatrix_df = pd.DataFrame(JaccardSimilarityMatrix, index=rowIndexList, columns=rowIndexList)
print(JaccardSimilarityMatrix_df)

          A    B      C         D      E
A  1.000000  0.5  0.375  0.285714  0.375
B  0.500000  1.0  0.600  0.500000  0.600
C  0.375000  0.6  1.000  0.600000  1.000
D  0.285714  0.5  0.600  1.000000  0.600
E  0.375000  0.6  1.000  0.600000  1.000


In [8]:
print(cosineSimilarityMatrix_df)
print(JaccardSimilarityMatrix_df)

          A         B         C         D         E
A  1.000000  0.707107  0.547723  0.471405  0.547723
B  0.707107  1.000000  0.774597  0.666667  0.774597
C  0.547723  0.774597  1.000000  0.774597  1.000000
D  0.471405  0.666667  0.774597  1.000000  0.774597
E  0.547723  0.774597  1.000000  0.774597  1.000000
          A    B      C         D      E
A  1.000000  0.5  0.375  0.285714  0.375
B  0.500000  1.0  0.600  0.500000  0.600
C  0.375000  0.6  1.000  0.600000  1.000
D  0.285714  0.5  0.600  1.000000  0.600
E  0.375000  0.6  1.000  0.600000  1.000


- 코사인 유사도에서는 각각의 사용자와 가장 유사한 상위 사용자 1명은  A - B / B - C&E / C - B&D / D - C & E / E - C
- 자카드 유사도에서는 각각의 사용자와 가장 유사한 상위 사용자 1명은  A - B / B - C&E / C - B&D / D - C & E / E - C